Кейс - есть набор данных клиентов, который состоит из 2 групп:
- приносящие прибыль (Exited = 1),
- не приносящие прибыль (Exited = 0)

Задача состоит в том, чтобы на данном наборе построить классификационную модель МО, которая сможет предсказывать на новых данных вероятность покупки клиентом услуги и оценить доходность стратегии развития бизнеса, состоящей из ежемесячных затрат в размере 1 доллара на yдержание клиента при учете того, что каждый лояльный клиент (TP) будет приносить 5 долларов в мес.

__Реализация__ 

Загружаем данные и пишем пайплайн обработки данных 

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
import numpy as np
import pandas as pd


from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from catboost import CatBoostClassifier 
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier

from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, KFold 

from sklearn.metrics import f1_score, roc_auc_score, log_loss, precision_recall_curve, confusion_matrix

%matplotlib inline

pd.set_option("display.max_rows", 6)

In [72]:
with open('/content/drive/MyDrive/ГБ/выборки для исследований/churn_data.csv', "rb") as f:
    df = pd.read_csv(f)
    
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [73]:
y = df['Exited']
X = df.drop(columns='Exited')

In [74]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [75]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
dtypes: float64(2), int64(8), object(3)
memory usage: 1015.8+ KB


'Tenure', 'HasCrCard', 'IsActiveMember' нужно перевести в тип данных объект

In [76]:
X = X.astype({'Tenure': np.object, 'HasCrCard': np.object, 'IsActiveMember': np.object})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [77]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [78]:
num_transfomer =  Pipeline([
            ('selector', ColumnSelector(key=continuous_columns)),
            ('standard', MinMaxScaler())
        ])

cat_transformer = Pipeline([
            ('selector', ColumnSelector(key=categorical_columns)),
            ('ohe', OHEEncoder(key=categorical_columns))
        ])
  
feats = FeatureUnion([('transfomer_num', num_transfomer),
                      ('cat_transformer', cat_transformer)
                      ])

#feature_processing = Pipeline([('sel', feats)])


In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)

Далее буду сравнивать работу 4 разных моделей (просто так захотелось): градиентный бустинг, логистическая регрессия, модель на методе опорных векторов и экстремальные деревья

In [80]:
# Пропишем алгоритм подсчета метрик для наших будущих моделей

results = pd.DataFrame(columns=['model', 'thresh', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])

def compute_result(model, preds_proba, results):
    precision, recall, thresholds = precision_recall_curve(y_test, preds_proba)
    fscore = (2 * precision * recall) / (precision + recall)
    auc = roc_auc_score(y_test, preds_proba)
    ix = np.argmax(fscore)

    results = results.append({
      'model': type(model['classifier']).__name__ ,
      'thresh': thresholds[ix],
      'F-Score': fscore[ix],
      'Precision': precision[ix],
      'Recall': recall[ix],
      'ROC AUC': auc
      }, ignore_index=True)
    
    return results



Все готово для тестирования разных моделей и поиска их лучших гиперпараметров.

<br>

__С точки зрения корректности отладки алгоритмов использование пайплайна в данном случае яв-ся совсем не оптимальным, т.к. при поиске лучшей модели он запускается каждый раз заново в новой модели. Оптимальным было бы сразу трансформировать X и уже его разбивать на трейн и тест. В данной работе практикуем работу с пайплайнами__

In [81]:
# Начнем с гр. бустинга
# %%time


# gb_model = Pipeline([
#     ('preprocessing', feats),
#     ('classifier', GradientBoostingClassifier(random_state=1))
# ])


# params = [{#'classifier__criterion': ['friedman_mse', 'squared_error', 'mse'], # долго все перебирать
#            'classifier__max_depth': [5,6], # range(4,7)
#            'classifier__n_estimators':  [50, 100, 300, 500], # 750, 1000
#            'classifier__learning_rate': [0.01, 0.1, 1], #, 10, 100
#            'classifier__min_samples_leaf': [1, 2, 4]
#            }]

# grid_gb = GridSearchCV(gb_model, params, cv=5, n_jobs=-1)
# grid_gb.fit(X_train, y_train)
# grid_gb.best_params_

# --------------------------
# CPU times: user 24.9 s, sys: 1.39 s, total: 26.3 s
# Wall time: 20min 56s
# {'classifier__learning_rate': 0.01,
#  'classifier__max_depth': 5,
#  'classifier__min_samples_leaf': 4,
#  'classifier__n_estimators': 500}


In [82]:
# Запишем пайплайн гб с лучшими гиперпараметрами и его метрики

gb_model = Pipeline([
    ('preprocessing', feats),
    ('classifier', GradientBoostingClassifier(random_state=1, learning_rate=0.01, max_depth=5, min_samples_leaf=4, n_estimators=500))
])

gb_model.fit(X_train, y_train)

pred_gb = gb_model.predict_proba(X_test)[:, 1]

results = compute_result(gb_model, pred_gb, results)


Теперь потестим логистическую регрессию:

In [83]:
# %%time

# model_lg = Pipeline([
#     ('preprocessing', feats),
#     ('classifier', LogisticRegression(random_state=1))
# ])

# params_grid = [{'classifier__C': [0.01, 0.1, 1, 10, 100],
#                'classifier__max_iter': [50, 100, 200, 300, 500],
#                'classifier__solver': ['lbfgs', 'liblinear']}
#                ]
                

# grid_lg = GridSearchCV(model_lg, params_grid, cv=5, n_jobs=-1)
# grid_lg.fit(X_train, y_train)
# grid_lg.best_params_

# ---------------------
# CPU times: user 5.53 s, sys: 297 ms, total: 5.83 s
# Wall time: 23 s
# {'classifier__C': 0.1,
#  'classifier__max_iter': 50,
#  'classifier__solver': 'lbfgs'}

In [84]:
# Запишем пайплайн лог. регрессии с лучшими гиперпараметрами и ее метрики

model_lg = Pipeline([
    ('preprocessing', feats),
    ('classifier', LogisticRegression(random_state=1,C=0.1, max_iter=50, solver='lbfgs'))
    ])

model_lg.fit(X_train, y_train)

pred_lg = model_lg.predict_proba(X_test)[:, 1]

results = compute_result(model_lg, pred_lg, results)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


Лог. регрессия на что-то ругается, думаю это связано с подчетом метрики fscore ...

Перейдем к модели на основе метода опорных векторов. Если в данных какой-то линейный косяк, который встретила лог. регрессия, то и на svc мы это должны увидеть

In [85]:
# %%time

# model_svc = Pipeline([
#     ('preprocessing', feats),
#     ('classifier', SVC(random_state=1))
# ])

# params_grid = [{'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
#                 'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]
#                 }]
                

# grid_svc = GridSearchCV(model_svc, params_grid, cv=5, n_jobs=-1)
# grid_svc.fit(X_train, y_train)
# grid_svc.best_params_


# #--------------------
# CPU times: user 8.77 s, sys: 647 ms, total: 9.41 s
# Wall time: 5min 12s
# {'classifier__C': 100, 'classifier__gamma': 0.1}

In [86]:
# Запишем пайплайн svc с лучшими гиперпараметрами и его метрики

model_svc = Pipeline([
    ('preprocessing', feats),
    ('classifier', SVC(random_state=1, C=100, gamma=0.1))
    ])

model_svc.fit(X_train, y_train)

pred_svc = model_svc.decision_function(X_test)

results = compute_result(model_svc, pred_svc, results)

Перейдем к модели экстремальных деревьев:

In [87]:
# %%time

# from sklearn.ensemble import ExtraTreesClassifier

# model_extr = Pipeline([
#     ('preprocessing', feats),
#     ('classifier', ExtraTreesClassifier(random_state=1))
#     ])



# params_grid = [{'classifier__n_estimators': [30, 50, 100, 150, 200],
#                 'classifier__criterion': ['gini', 'entropy'],
#                 'classifier__min_samples_leaf': [1,2,4,8]
#                 }]
                

# grid_extr = GridSearchCV(model_extr, params_grid, cv=5, n_jobs=-1)
# grid_extr.fit(X_train, y_train)
# grid_extr.best_params_


# # ---------------------
# CPU times: user 5.67 s, sys: 286 ms, total: 5.96 s
# Wall time: 1min 49s
# {'classifier__criterion': 'entropy',
#  'classifier__min_samples_leaf': 4,
#  'classifier__n_estimators': 100}

In [88]:
# Запишем модель с лучшими парметрами

from sklearn.ensemble import ExtraTreesClassifier

model_extr = Pipeline([
    ('preprocessing', feats),
    ('classifier', ExtraTreesClassifier(criterion='entropy', min_samples_leaf=4, random_state=1)) # n_estimators по умолчанию итак 100
    ])

model_extr.fit(X_train, y_train)

preds = model_extr.predict_proba(X_test)[:, 1]
results = compute_result(model_extr, preds, results)

Выведем таблицу сравнения метрик

In [89]:
results


,model,thresh,F-Score,Precision,Recall,ROC AUC
0,GradientBoostingClassifier,0.385376,0.646483,0.731855,0.578947,0.870031
1,LogisticRegression,0.842417,NaN,0.000000,0.000000,0.756649
2,SVC,-0.423594,0.598383,0.685185,0.531100,0.818572
3,ExtraTreesClassifier,0.323107,0.609477,0.624791,0.594896,0.842716


Видим что с логистической регрессией что-то не то, при этом svc отработал корректно.

Лучше всего отрабатывает Градиентный бустинг, т.к. имеем самые высокие fscore и roc auc

__Теперь приступим к оценке прибыльности бизнес-стратегии:__ 
<br>

Для начала нужно получить confusion matrix, а перед этим сформировать предсказания классов у нашей модели гр. бустинга, исходя из найденного порога 0.385376.


In [105]:
# повторим чтобы было перед глазами. Предсказания лучшей модели гр. бустинга
pred_gb = gb_model.predict_proba(X_test)[:, 1]

# вариант 1
# cnf_matrix = confusion_matrix(y_test, pred_gb > 0.385376)

# TN = cnf_matrix[0][0]
# FP = cnf_matrix[0][1]
# FN = cnf_matrix[1][0]
# TP = cnf_matrix[1][1]

# вариант 2
TN, FP, FN, TP = confusion_matrix(y_test, pred_gb > 0.385376).ravel() # последовательность TN, FP, FN, TP из документации

# затраты на удержание
retain_sum = (FP + TP) * 1   # перерасход при удержании идет за счет тех, кого считаем положительными и ошибаемся в этом

# доход с верно определенного лояльным клиента
income = TP * 5      # доход получаем только с выявленного по-настоящему лояльного клиента

# прибыльность бизнес стратегии тратить бакс на удержание при доходе в два бакса с удержанного настоящего лояльного
print(f' Доходность бизнес-стратегии: {income - retain_sum}$ в мес. чистыми, расходы на неверно классифицированных клиентов {FP * 1}$')

 Доходность бизнес-стратегии: 1319$ в мес. чистыми, расходы на неверно классифицированных клиентов 133$


In [106]:
TP

363

Таким образом, данная модель МО и выбранная бизнес стратегия позволяют приносить ежемесячно 1319$


Сравним ради интереса показатель доходности с неулучшенной моделью МО (настройки по умолчанию) и без подбора порога:

In [107]:
gb_model_poor = Pipeline([
    ('preprocessing', feats),
    ('classifier', GradientBoostingClassifier(random_state=1))
])

gb_model_poor.fit(X_train, y_train)

pred_gb_poor = gb_model_poor.predict(X_test)

TN, FP, FN, TP = confusion_matrix(y_test, pred_gb_poor).ravel() # последовательность TN, FP, FN, TP из документации

# затраты на удержание
retain_sum = (FP + TP) * 1   

# доход с верно определенного лояльным клиента
income = TP * 5     

# прибыльность бизнеса
print(f' Доходность бизнес-стратегии: {income - retain_sum}$ в мес. чистыми, расходы на неверно классифицированных клиентов {FP * 1}$')

 Доходность бизнес-стратегии: 1100$ в мес. чистыми, расходы на неверно классифицированных клиентов 84$


In [108]:
TP

296

Видно что тюнинговая модель с оптимально выбранным порогом позволяет больше выявлять реальных лояльных клиентов, при больше получаем и ложных лояльных клиентов и затраты на них, однако чистых доход эта модель приносит на 219 баксов больше 

In [109]:
pd.crosstab(y_test, pred_gb > 0.385376)

col_0,False,True
Exited,,
0,2240,133
1,264,363


In [110]:
pd.crosstab(y_test, pred_gb_poor)

col_0,0,1
Exited,,
0,2289,84
1,331,296
